## **Príklady: Multi-AI agenti na rezerváciu hotelov**

V dnešnom rýchlom svete plánovanie pracovnej cesty zahŕňa viac než len rezerváciu letu a hotelovej izby. Vyžaduje si úroveň koordinácie a efektivity, ktorú môže byť náročné dosiahnuť. Tu prichádzajú na scénu Multi-AI agenti, ktorí revolučne menia spôsob, akým spravujeme naše cestovné potreby.

Predstavte si tím inteligentných agentov, ktorí sú vám k dispozícii a spolupracujú na tom, aby zvládli každý aspekt vašej cesty s presnosťou a ľahkosťou. Vďaka našej pokročilej AI technológii sme vytvorili špecializovaných agentov na rezerváciu služieb a plánovanie itinerára, čím zabezpečujeme plynulý a bezstresový zážitok z cestovania.

Toto je základný scenár. Pri plánovaní pracovnej cesty je potrebné konzultovať s agentom pre pracovné cesty, aby sme získali informácie o letenkách, hoteloch a podobne. Prostredníctvom AI agentov môžeme vytvoriť agentov na rezerváciu služieb a agentov na plánovanie itinerára, ktorí spolupracujú a zvyšujú úroveň inteligencie.


# Inicializácia služby Azure AI Agent a získanie konfiguračných informácií z **.env**

### **.env** 

Vytvorte súbor .env 

**.env** obsahuje reťazec pripojenia služby Azure AI Agent, model používaný AOAI a zodpovedajúcu službu Google API Search, ENDPOINT, atď.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Názov nasadenia modelu služby Azure AI Agent"

[**NOTE**] Budete potrebovať model s limitom 100 000 (tokeny za minútu) a limitom 600 (požiadavky za minútu).

  Model môžete získať v Azure AI Foundry - Model a Endpoint. 

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Reťazec pripojenia projektu služby Azure AI Agent"

  Reťazec pripojenia projektu môžete získať v prehľade projektu na obrazovke AI ​​Foundry Portálu.

- **SERPAPI_SEARCH_API_KEY** = "Váš SERPAPI Search API KEY"
- **SERPAPI_SEARCH_ENDPOINT** = "Váš SERPAPI Search Endpoint"

Na získanie názvu nasadenia modelu a reťazca pripojenia projektu služby Azure AI Agent je potrebné vytvoriť službu Azure AI Agent. Odporúča sa použiť [túto šablónu](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) na jej priame vytvorenie (***Poznámka:*** Služba Azure AI Agent je momentálne dostupná len v obmedzených regiónoch. Odporúča sa, aby ste si pozreli [tento odkaz](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) na nastavenie regiónu).

Agent potrebuje prístup k SERPAPI. Odporúča sa zaregistrovať sa pomocou [tohto odkazu](https://serpapi.com/searches). Po registrácii môžete získať unikátny API KEY a ENDPOINT.


# Prihlásenie do Azure

Teraz sa musíte prihlásiť do Azure. Otvorte terminál vo VScode a spustite príkaz `az login`.


# Nastavenie

Na spustenie tohto notebooku budete potrebovať nainštalovať nasledujúce knižnice. Tu je zoznam potrebných knižníc a zodpovedajúcich príkazov na inštaláciu pomocou pip:

azure-identity: Na autentifikáciu v Azure.  
requests: Na vykonávanie HTTP požiadaviek.  
semantic-kernel: Pre rámec semantic kernel (za predpokladu, že ide o vlastnú alebo špecifickú knižnicu, možno ju budete musieť nainštalovať zo špecifického zdroja alebo repozitára).  


In [ ]:
!pip install azure-identity
!pip install requests
!pip install semantic-kernel
!pip install --upgrade semantic_kernel
!pip install azure-cli

# Vysvetlenie: 
import asyncio: Týmto sa importuje modul asyncio, ktorý poskytuje podporu pre asynchrónne programovanie v Pythone. Umožňuje písať súbežný kód pomocou syntaxe async a await.
from typing import Annotated: Týmto sa importuje typ Annotated z modulu typing. Annotated sa používa na pridanie metaúdajov k typovým anotáciám, čo môže byť užitočné na rôzne účely, ako napríklad validácia, dokumentácia alebo nástroje.


In [ ]:
import asyncio,os
from typing import Annotated

# Vysvetlenie:
Použitím from dotenv import load_dotenv a load_dotenv() môžete jednoducho spravovať konfiguračné nastavenia a citlivé informácie (ako API kľúče a URL databáz) v súbore .env, čím ich oddelíte od zdrojového kódu. Týmto spôsobom spravíte svoju aplikáciu bezpečnejšou a jednoduchšou na konfiguráciu.


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Vysvetlenie:

Importovanie: from azure.identity.aio import DefaultAzureCredential: Tento príkaz importuje triedu DefaultAzureCredential z modulu azure.identity.aio. Časť názvu modulu "aio" naznačuje, že je určený pre asynchrónne operácie.

Účel DefaultAzureCredential: Trieda DefaultAzureCredential je súčasťou Azure SDK pre Python. Poskytuje predvolený spôsob autentifikácie s Azure službami. Pokúša sa autentifikovať pomocou viacerých metód v konkrétnom poradí, ako sú environmentálne premenné, spravovaná identita a Azure CLI poverenia.

Asynchrónne operácie: Modul "aio" naznačuje, že trieda DefaultAzureCredential podporuje asynchrónne operácie. To znamená, že ju môžete použiť s asyncio na vykonávanie neblokujúcich autentifikačných požiadaviek.


In [ ]:
from azure.identity.aio import DefaultAzureCredential

# Vysvetlenie:
Importuje rôzne moduly a triedy z balíka semantic_kernel. Tu je rozpis jednotlivých importov:

AgentGroupChat zo semantic_kernel.agents: Táto trieda spracováva funkcie súvisiace so skupinovým chatom pre AI agentov. AzureAIAgent a AzureAIAgentSettings zo semantic_kernel.agents.azure_ai

AzureAIAgent: Táto trieda sa používa na vytváranie a správu AI agentov, ktorí využívajú služby Azure AI.

AzureAIAgentSettings: Táto trieda slúži na konfiguráciu nastavení pre AzureAIAgent. TerminationStrategy zo semantic_kernel.agents.strategies.termination.termination_strategy:

Táto trieda definuje stratégie na ukončenie vykonávania AI agentov za určitých podmienok. ChatMessageContent zo semantic_kernel.contents.chat_message_content:

Táto trieda sa používa na spracovanie obsahu chatových správ.
AuthorRole zo semantic_kernel.contents.utils.author_role:

Táto trieda definuje rôzne roly autorov v kontexte chatových správ. 

kernel_function zo semantic_kernel.functions.kernel_function_decorator: Tento dekorátor sa používa na definovanie kernel funkcií, čo sú funkcie, ktoré môžu byť vykonávané v rámci frameworku semantic kernel.
Tieto importy nastavujú potrebné komponenty na vytváranie a správu AI agentov, ktorí môžu interagovať v prostredí skupinového chatu, pravdepodobne na úlohy ako rezervácia hotelov alebo podobné aktivity.


In [ ]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.contents import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# Vysvetlenie:
Ďalej importujeme triedu CodeInterpreterTool z modulu azure.ai.projects.models.

CodeInterpreterTool: Táto trieda je súčasťou Azure AI SDK a slúži na interpretáciu a vykonávanie kódu v rámci AI projektov. Poskytuje funkcie na spúšťanie útržkov kódu, analýzu kódu alebo integráciu vykonávania kódu do AI pracovných postupov. Tento import nastavuje potrebnú súčasť na využitie CodeInterpreterTool vo vašom projekte, čo môže byť užitočné pre úlohy zahŕňajúce dynamickú interpretáciu a vykonávanie kódu.


In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

# Vysvetlenie: 
Trieda ApprovalTerminationStrategy poskytuje konkrétnu stratégiu na ukončenie operácie AI agenta. Agent sa ukončí, ak posledná správa v jeho histórii interakcií obsahuje slovo "uložené". To môže byť užitočné v situáciách, kde sa úloha agenta považuje za dokončenú, keď dostane potvrdenie, že niečo bolo "uložené". Definujte metódu interakcie. Po uložení plánu rezervácie môže byť zastavený pri prijatí signálu "uložené".


In [ ]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "saved" in history[-1].content.lower()

# Vysvetlenie:

Tento riadok kódu inicializuje objekt AzureAIAgentSettings s predvolenými alebo vopred definovanými nastaveniami pomocou metódy create(). Tento objekt nastavení (ai_agent_settings) môže byť následne použitý na konfiguráciu a správu inštancie AzureAIAgent.


In [ ]:
ai_agent_settings = AzureAIAgentSettings.create()

# Vysvetlenie:
Importovaním knižnice requests môžete jednoducho vykonávať HTTP požiadavky a pracovať s webovými službami vo vašom Python kóde.


In [ ]:
import requests

# Vysvetlenie:
Toto je premenná, ktorá ukladá API kľúč na prístup k službe SERP (Search Engine Results Page). API kľúč je jedinečný identifikátor používaný na autentifikáciu požiadaviek spojených s vaším účtom.

'GOOGLE_SEARCH_API_KEY': Toto je zástupný reťazec. Musíte nahradiť 'GOOGLE_SEARCH_API_KEY' vaším skutočným API kľúčom pre SERP.

Účel: Účelom tohto riadku je uložiť API kľúč do premennej, aby sa mohol použiť na autentifikáciu požiadaviek na službu SERP API. API kľúč je potrebný na prístup k službe a vykonávanie vyhľadávaní.

Ako získať API kľúč pre SERP: Ak chcete získať API kľúč pre SERP, postupujte podľa týchto všeobecných krokov na https://serpapi.com (presné kroky sa môžu líšiť v závislosti od konkrétnej služby SERP API, ktorú používate):

Vyberte si službu SERP API: Existuje niekoľko služieb SERP API, ako napríklad SerpAPI, Google Custom Search JSON API a ďalšie. Vyberte si tú, ktorá najlepšie vyhovuje vašim potrebám.

Zaregistrujte si účet:

Prejdite na webovú stránku vybranej služby SERP API https://www.serpapi.com a zaregistrujte si účet. Možno budete musieť poskytnúť základné informácie a overiť svoju e-mailovú adresu.

Vytvorte API kľúč:

Po registrácii sa prihláste do svojho účtu a prejdite do sekcie API alebo na ovládací panel. Hľadajte možnosť vytvoriť alebo vygenerovať nový API kľúč.
Skopírujte API kľúč:

Keď je API kľúč vygenerovaný, skopírujte ho. Tento kľúč sa bude používať na autentifikáciu vašich požiadaviek na službu SERP API.
Nahraďte zástupný reťazec:

Nahraďte zástupný reťazec vo vašom súbore .env.


In [ ]:
SERPAPI_SEARCH_API_KEY=os.getenv('SERPAPI_SEARCH_API_KEY')

In [ ]:
SERPAPI_SEARCH_ENDPOINT = os.getenv('SERPAPI_SEARCH_ENDPOINT')

# Vysvetlenie:
Trieda BookingPlugin poskytuje metódy na rezerváciu hotelov a letov pomocou Google Search API od Serpapi.com. Vytvára potrebné parametre, odosiela požiadavky na API a spracováva odpovede, aby vrátila relevantné informácie o rezerváciách. API kľúč (SERPAPI_SEARCH_API_KEY) a endpoint (SERPAPI_SEARCH_ENDPOINT) sa používajú na autentifikáciu a odosielanie požiadaviek na Google Search API.


In [ ]:
# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""
    @kernel_function(description="booking hotel")
    def booking_hotel(self,query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-in Time"])-> Annotated[str, "Return the result of booking hotel infomation"]:

        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "2",
            "currency": "USD",
            "gl": "us",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY
        }

        response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=params)
        if response.status_code == 200:
            response = response.json()
            return response["properties"]
        else:
            return None

    
    @kernel_function(description="booking fight")
    def  booking_fight(self,origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"])-> Annotated[str, "Return the result of booking fight infomation"]:
        
        go_params = {
            "engine": "google_flights",   
            "departure_id": origin,
            "arrival_id": destination,
            "outbound_date": outbound_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }

        print(go_params)

        go_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=go_params)


        result = ''

        if go_response.status_code == 200:
            response = go_response.json()

            result += "# outbound \n " + str(response)
        else:
            print('error!!!')
            # return None

        
        back_params = {
            "engine": "google_flights",   
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": return_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }


        print(back_params)


        back_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=back_params)



        if back_response.status_code == 200:
            response = back_response.json()

            result += "\n # return \n"  + str(response)

        else:
            print('error!!!')
            # return None
        
        print(result)

        return result

        


# Vysvetlenie:
Trieda SavePlugin poskytuje metódu saving_plan na uloženie plánov ciest pomocou služieb Azure AI. Nastavuje Azure poverenia, vytvára AI agenta, spracováva vstupy používateľa na generovanie a uloženie obsahu plánu cesty a zabezpečuje operácie ukladania súborov a čistenia. Metóda vráti "Saved" po úspešnom dokončení.


In [ ]:
class SavePlugin:
    """Save Plugin for customers"""
    @kernel_function(description="saving plan")
    async def saving_plan(self,tripplan: Annotated[str, "The content of trip plan"])-> Annotated[str, "Return status of save content"]:

        async with (
            DefaultAzureCredential() as creds,
            AzureAIAgent.create_client(
                credential=creds,
                conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
            ) as client,
        ):

            code_interpreter = CodeInterpreterTool()
            
            agent_definition = await client.agents.create_agent(
                model=ai_agent_settings.model_deployment_name,
                tools=code_interpreter.definitions,
                tool_resources=code_interpreter.resources,
            )


            agent = AzureAIAgent(
                client=client,
                definition=agent_definition,
            )

            thread = await client.agents.create_thread()


            user_inputs = [
                """
            
                        You are my Python programming assistant. Generate code,save """+ tripplan +
                        
                    """    
                        and execute it according to the following requirements

                        1. Save blog content to trip-{YYMMDDHHMMSS}.md

                        2. give me the download this file link
                    """
            ]



            try:
                for user_input in user_inputs:
                    # Add the user input as a chat message
                    await agent.add_chat_message(
                        thread_id=thread.id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
                    )
                    print(f"# User: '{user_input}'")
                    # Invoke the agent for the specified thread
                    async for content in agent.invoke(thread_id=thread.id):
                        if content.role != AuthorRole.TOOL:
                            print(f"# Agent: {content.content}")

                    
                    messages = await client.agents.list_messages(thread_id=thread.id)

                    # OpenAIPageableListOfThreadMessage
                    # OpenAIPageableListOfThreadMessage


                    for file_path_annotation in messages.file_path_annotations:

                            file_name = os.path.basename(file_path_annotation.text)

                            await client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./trip")

                    
            finally:
                await client.agents.delete_thread(thread.id)
                await client.agents.delete_agent(agent.id)


        return "Saved"

# Vysvetlenie:
Tento kód nastavuje agentov Azure AI na spracovanie rezervácií letov a hotelov a ukladanie plánov ciest na základe vstupov od používateľov. Používa Azure poverenia na vytvorenie a konfiguráciu agentov, spracováva vstupy používateľov prostredníctvom skupinového chatu a zabezpečuje správne vyčistenie po dokončení úloh. Agenti používajú špecifické pluginy (BookingPlugin a SavePlugin) na vykonávanie svojich príslušných úloh.


In [ ]:
async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):
    BOOKING_AGENT_NAME = "BookingAgent"
    BOOKING_AGENT_INSTRUCTIONS = """
    You are a booking agent. Help me book flights or hotels.

    Thought: Please understand the user's intention and confirm whether to use the reservation system to complete the task.

    Actions:
    - For flight bookings, convert the departure and destination names into airport codes.
    - Use the appropriate API for hotel or flight bookings. Verify that all necessary parameters are available. If any parameters are missing, ask the user to provide them. If all parameters are complete, call the corresponding function.
    - If the task is not related to hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
      - For flight bookings, output separate outbound and return contents in the order of:
        Departure Airport | Airline | Flight Number | Departure Time | Arrival Airport | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
      - For hotel bookings, output in the order of:
        Property Name | Property Description | Check-in Time | Check-out Time | Prices | Nearby Places | Hotel Class | GPS Coordinates.
    """

    SAVE_AGENT_NAME = "SaveAgent"
    SAVE_AGENT_INSTRUCTIONS = """
    You are a save tool agent. Help me to save the trip plan.
    """

    # Create agent definition
    booking_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=BOOKING_AGENT_NAME,
        instructions=BOOKING_AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent
    booking_agent = AzureAIAgent(
        client=client,
        definition=booking_agent_definition,
        # Optionally configure polling options
        # polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
    )

    # Add the sample plugin to the kernel
    booking_agent.kernel.add_plugin(BookingPlugin(), plugin_name="booking")

    # Create agent definition
    save_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=SAVE_AGENT_NAME,
        instructions=SAVE_AGENT_INSTRUCTIONS
    )

    # Create the AzureAI Agent
    save_agent = AzureAIAgent(
        client=client,
        definition=save_agent_definition,
    )

    save_agent.kernel.add_plugin(SavePlugin(), plugin_name="saving")

    user_inputs = [
        "I have a business trip from London to New York in Feb 20 2025 to Feb 27 2025 ,help me to book a hotel and fight tickets and save it"
    ]

    chat = AgentGroupChat(
        agents=[booking_agent, save_agent],
        termination_strategy=ApprovalTerminationStrategy(agents=[save_agent], maximum_iterations=10),
    )

    try:
        for user_input in user_inputs:
            # Add the user input as a chat message
            await chat.add_chat_message(
                ChatMessageContent(role=AuthorRole.USER, content=user_input)
            )
            print(f"# User: '{user_input}'")

            async for content in chat.invoke():
                print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

            print(f"# IS COMPLETE: {chat.is_complete}")

            print("*" * 60)
            print("Chat History (In Descending Order):\n")
            async for message in chat.get_chat_messages(agent=save_agent):
                print(f"# {message.role} - {message.name or '*'}: '{message.content}'")
    finally:
        await chat.reset()
        await client.agents.delete_agent(save_agent.id)
        await client.agents.delete_agent(booking_agent.id)



---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby AI prekladu [Co-op Translator](https://github.com/Azure/co-op-translator). Hoci sa snažíme o presnosť, prosím, berte na vedomie, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho pôvodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre kritické informácie sa odporúča profesionálny ľudský preklad. Nie sme zodpovední za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
